In [1]:
from dask.distributed import Client

client = Client("tls://192.168.235.17:8786")
client

/usr/local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Connection method: Direct,
Dashboard: /user/mohammad.sahraei@cern.ch/proxy/8787/status,
Comm: tls://192.168.235.17:8786,Workers: 0
Dashboard: /user/mohammad.sahraei@cern.ch/proxy/8787/status,Total threads: 0
Started: 6 days ago,Total memory: 0 B


In [2]:
import hist
import dask
import awkward as ak
import hist.dask as hda
import dask_awkward as dak

import numpy as np
import json

from coffea import processor
from coffea.analysis_tools import PackedSelection
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
import vector
from coffea.nanoevents.methods import candidate
from coffea.dataset_tools import (
    apply_to_fileset,
    max_chunks,
    preprocess,
)

/usr/local/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [3]:
with open("fileset.json", "r") as fileset_data:
    fileset = json.load(fileset_data)

In [4]:
from dask.distributed import performance_report

with performance_report(filename="dask-report.html"):
    dataset_runnable, dataset_updated = preprocess(
        fileset,
        align_clusters=False,
        step_size=1000000,
        files_per_batch=1,
        skip_bad_files=False,
        save_form=False,
    )

In [5]:
class MyProcessor(processor.ProcessorABC):
    def __init__(self):
        pass

    def process(self, events):
        
        selection = PackedSelection()
        selection.add_multiple(
            {
                'photon_trigger' : events.HLT.Photon175 == True,
                'photon_selection': dak.num(events.Photon, axis=1) == 1,
                'photon_pt': dak.all(events.Photon.pt > 200 , axis=1),
                'photon_eta': dak.all(np.abs(events.Photon.eta) < 1.44, axis=1),
                'no_electron': dak.num(events.Electron) == 0,
                'no_muon': dak.num(events.Muon) == 0,
                'jet_selection': dak.num(events.Jet, axis=1) == 3,
                'jet_pt': dak.all(events.Jet.pt > 20 , axis=1),
                'jet_eta': dak.all(np.abs(events.Jet.eta) < 2.5, axis=1),
                'btag_jet': (dak.num(events.Jet[events.Jet.btagCSVV2 > 0.5426])==1)
            }
        )
        
        print(selection.cutflow('jet_selection', 'photon_trigger', 'photon_selection', 'photon_pt',
                                'no_electron', 'no_muon', 'jet_pt', 'jet_eta','btag_jet' 
                               ).print(), events.metadata['dataset'])
        
        return {
#             dataset: {
                
#             }
        }

    def postprocess(self, accumulator):
        pass

In [6]:
from dask.distributed import performance_report

with performance_report(filename="dask-report.html"):
    to_compute = apply_to_fileset(
        MyProcessor(),
        dataset_runnable,
        schemaclass=NanoAODSchema,
    )

/usr/local/lib/python3.10/site-packages/coffea/analysis_tools.py:866: UserWarning: Printing the cutflow statistics is going to compute dask_awkward objects.
  warnings.warn(


Cutflow stats:
Cut jet_selection       :pass = 10170147            cumulative pass = 10170147            all = 56878553            -- eff = 17.9 %                    -- cumulative eff = 17.9 %
Cut photon_trigger      :pass = 6354164             cumulative pass = 777255              all = 56878553            -- eff = 11.2 %                    -- cumulative eff = 1.4 %
Cut photon_selection    :pass = 37318345            cumulative pass = 483288              all = 56878553            -- eff = 65.6 %                    -- cumulative eff = 0.8 %
Cut photon_pt           :pass = 6561168             cumulative pass = 374268              all = 56878553            -- eff = 11.5 %                    -- cumulative eff = 0.7 %
Cut no_electron         :pass = 30751298            cumulative pass = 102208              all = 56878553            -- eff = 54.1 %                    -- cumulative eff = 0.2 %
Cut no_muon             :pass = 47804775            cumulative pass = 80329               all = 568

OSError: File did not open properly: [ERROR] Server responded with an error: [3010] Unable to open /store/data/Run2018D/EGamma/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v3/250000/EE3D3644-112A-DC47-8641-8848C10A90E5.root; permission denied


In [9]:
from dask.distributed import performance_report

with performance_report(filename="dask-report.html"):
    (out,) = dask.compute(to_compute)

In [ ]:
[ERROR] Server responded with an error: [3010] Unable to open /store/data/Run2018D/EGamma/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v3/250000/1F63C3E2-9D89-AE49-96B3-25BE99867F39.root; permission deni

In [10]:
out

{'/SinglePhoton/Run2016B-ver2_HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD': {},
 '/SinglePhoton/Run2016C-HIPM_UL2016_MiniAODv2_NanoAODv9-v4/NANOAOD': {},
 '/SinglePhoton/Run2016D-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD': {},
 '/SinglePhoton/Run2016E-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD': {},
 '/SinglePhoton/Run2016F-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD': {},
 '/SinglePhoton/Run2016F-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/SinglePhoton/Run2016G-UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD': {},
 '/SinglePhoton/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/SinglePhoton/Run2017B-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/SinglePhoton/Run2017C-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/SinglePhoton/Run2017D-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/SinglePhoton/Run2017E-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/SinglePhoton/Run2017F-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/EGamma/Run2018A-UL2018_MiniAODv2_NanoAODv9-v1/NANOAOD': {},
 '/EGamma/